## Fixing those pesky Airmass Headings

In [1]:
import numpy as np
import glob
from ccdproc import CCDData

In [11]:
# Read in a list of all your data files 
data_list = glob.glob('C:/Users/josue/Work-space/Project 2/Data/07.31.2022/*.fits')

In [12]:
# File: the fits file whose airmass and zenith angle are incorrect
# Overwrites header with freshly calculated zenith angle and airmass
# ignore the info statement (MRO headers have the unit as "'adu'" not just "adu" so its annoying)

def airmassFix(file):
    image = CCDData.read(file,unit='adu')
    header = image.header
    # Turns out numpy uses radians
    mro_lat = header['LATITUDE'] *np.pi/180.0

    dec = header['DEC'] *np.pi/180.0
    
    ha = header['HA'] * 15.0 *np.pi/180.0

    za = np.arccos(np.sin(mro_lat)*
                  np.sin(dec)+
                  np.cos(mro_lat)*
                  np.cos(dec)*
                  np.cos(ha))
    
    airmass = 1.0/np.cos(za)
    
    image.header['ZD'] = za * 180.0 /np.pi
    image.header['AIRMASS'] = np.abs(airmass)
    image.write(file,image.header,overwrite=True);

In [13]:
for file in data_list:
    airmassFix(file)